In this notebook we first do the classification using the transformer This is our first classification task.

The output classification vector from the transformer is saved to be used by the FCNN This is our second classification task.


In [1]:
# Importing necessary libraries
import pandas as pd
from datetime import datetime
import sklearn
import torch
import torch.nn as nn
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [12]:
from simpletransformers.classification import ClassificationModel

## Preparing the dataset

Some pre-processing to the dataset has already been done in preparation for various tests, so this processing is not from scratch.

In [4]:
# procedure for getting the data sets and formatting them for the transformer
 

def prepareDataset( filename):
     
    ReadSet=pd.read_excel(filename )

    ReadSet['text']=ReadSet['Statement']
    ReadSet['labels']=ReadSet['Label']
    
    ReadSet=ReadSet.drop(['ID','Label','Statement','Subject','Speaker','Job','From','Affiliation','PantsTotal','NotRealTotal','BarelyTotal','HalfTotal','MostlyTotal','Truths','Context'
],axis=1)
     
    return ReadSet


In [5]:
# preparing the training dataset
train=prepareDataset( 'trainRNDtext.xlsx')
# and display for inspecting
train

,text,labels
0,It costs more money to put a person on death r...,0
1,"Charlie Crist ""bizarrely vetoed"" $9.7 million ...",3
2,"In Massachusetts, half of the primary care doc...",4
3,Fulton County has successfully reduced the num...,4
4,Says he stood up to his own party by voting ag...,5
...,...,...
10264,Recently Rick Scott closed 30 womens health ca...,4
10265,Says Charlie Bass supports Paul Ryan plan that...,5
10266,A report by the US General Accountability Offi...,3
10267,"In the United States, weve had 12 years in a r...",1


In [6]:
# preparing the evaluation/validation dataset
Eval=prepareDataset('valid.xlsx')
# and display for inspecting
Eval

,text,labels
0,The president is brain-dead.,0
1,"Barack Obama supported keeping troops in Iraq,...",3
2,"He's leading by example, refusing contribution...",3
3,I'm the first person who really took up the is...,4
4,I built that border fence in San Diego...and i...,4
...,...,...
1279,CNN accidentally aired 30 minutes of pornograp...,1
1280,President Obamas American Recovery and Reinves...,2
1281,We (in Illinois) have the fifth-highest tax bu...,4
1282,Says Donald Trump won more counties than any c...,4


In [7]:
# preparing the test set dataset
test=prepareDataset('test.xlsx')
test

,text,labels
0,New Mexico was 46th in teacher pay (when he wa...,4
1,Barack Obama and Hillary Clinton have changed ...,3
2,I'll tell you what I can tell this country: If...,1
3,Tommy Thompson created the first school choice...,5
4,Fifty-six percent decline in overall crime. A ...,5
...,...,...
1278,"We have trade agreements with 20 countries, an...",1
1279,On Donald Trumps plan to cut federal funding t...,4
1280,"Black Lives Matter, who are attacking law enfo...",2
1281,Latina who enthusiastically supported Donald T...,0


## Setting up the transformer for fine tuning

This is where changes are done to optimise the model

The simpletransformers library is the quickest way to do this at the time of writing. 
For more information on the settings and their default value go here:
https://github.com/ThilinaRajapakse/simpletransformers#default-settings 

###### Please do read that reference before changing any parameters. Don't try to be a hero!

In [8]:
#Set the model being used here
model_class='roberta'  # bert or roberta or albert
model_version='roberta-large' #bert-base-cased, roberta-base, roberta-large, albert-base-v2 OR albert-large-v2


output_folder='./TunedModels/'+model_class+'/'+model_version+"/"
cache_directory= "./TunedModels/"+model_class+"/"+model_version+"/"+"/cache/"
labels_count=6  # the number of classification classes

print('model variables were set up: ')

model variables were set up: 


In [9]:
# use this to test if writing to the directories is working

import os
print(os.getcwd())
print(output_folder)
print(cache_directory)

testWrite=train.head(30)
 
testWrite.to_csv(output_folder+'DeleteThisToo.tsv', sep='\t')
testWrite.to_csv(cache_directory+'DeleteThisToo.tsv', sep='\t')

del(testWrite)

G:\0 finalThesis\randomised LIAR
./TunedModels/roberta/roberta-large/
./TunedModels/roberta/roberta-large//cache/


In [13]:
 
save_every_steps=1285
# assuming training batch size of 8
# any number above 1284 saves the model only at every epoch
# Saving the model mid training very often will consume disk space fast

train_args={
    "output_dir":output_folder,
    "cache_dir":cache_directory,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'num_train_epochs': 2,
    "save_steps": save_every_steps, 
    "learning_rate": 2e-5,
    "train_batch_size": 64,
    "eval_batch_size": 32,
    "adam_epsilon": 1e-8,
    "evaluate_during_training_steps": 5,
    "max_seq_length": 64,
    "n_gpu": 1,
}

# Create a ClassificationModel
model = ClassificationModel(model_class, model_version, num_labels=labels_count, args=train_args) 

# You can set class weights by using the optional weight argument

### Loading a saved model (based on above args{})

If you stopped training you can continue training from a previously saved check point.
The next cell allows you to load a model from any checkpoint.
The number of epochs in the train_args{} will be done and continue tuning from your checkpoint.

###### HOWEVER
It will overwrite previous checkpoints!
Example:  If you load an epoch-3 checkpoint, the epoch-1 checkpoint will be overwritten by the 4th epoch and it will be equivalent to a 4th epoch even if you have epoch-1 in the name.
###### SO BE CAREFUL

In [14]:
# loading a previously saved model based on this particular Transformer Class and model_name

# loading the checkpoint that gave the best result
CheckPoint='checkpoint-161-epoch-1'


preSavedCheckpoint=output_folder+CheckPoint

print('Loading model, please wait...')
model = ClassificationModel( model_class, preSavedCheckpoint, num_labels=labels_count, args=train_args) 
print('model in use is :', preSavedCheckpoint )

Loading model, please wait...
model in use is : ./TunedModels/roberta/roberta-large/checkpoint-161-epoch-1


## Training the Transformer

Skip the next cell if you want to skip the training and go directly to the evaluation

In [10]:
# Train the model
current_time = datetime.now()
model.train_model(train)
print("Training time: ", datetime.now() - current_time)

Converting to features started. Cache is not used.



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 1.908411

C:\Users\Mark\Anaconda3\envs\thesis\lib\site-packages\torch\optim\lr_scheduler.py:110: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Seems like `optimizer.step()` has been overridden after learning rate scheduler "


Running loss: 1.746816

C:\Users\Mark\Anaconda3\envs\thesis\lib\site-packages\torch\optim\lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 1.724902Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 1.791989Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Running loss: 1.768177

Training of roberta model complete. Saved to ./TunedModels/roberta/roberta-large/.
Training time:  0:06:20.064507


## Evaluating the training

In [17]:
TrainResult, TrainModel_outputs, wrong_predictions = model.eval_model(train, acc=sklearn.metrics.accuracy_score)

EvalResult, EvalModel_outputs, wrong_predictions = model.eval_model(Eval, acc=sklearn.metrics.accuracy_score)

print('Training Result:', TrainResult['acc'])
#print('Model Out:', TrainModel_outputs)

print('Eval Result:', EvalResult['acc'])
#print('Model Out:', EvalModel_outputs)

Converting to features started. Cache is not used.



{'mcc': 0.0, 'acc': 0.20673872821112085, 'eval_loss': 1.7572507394065737}
Converting to features started. Cache is not used.


C:\Users\Mark\Anaconda3\envs\thesis\lib\site-packages\sklearn\metrics\_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)



{'mcc': 0.0, 'acc': 0.19314641744548286, 'eval_loss': 1.7617364336804646}
Training Result: 0.20673872821112085
Eval Result: 0.19314641744548286


In [18]:
TestResult, TestModel_outputs, wrong_predictions = model.eval_model(test, acc=sklearn.metrics.accuracy_score)

print('Test Set Result:', TestResult['acc'])
#print('Model Out:', TestModel_outputs)

Converting to features started. Cache is not used.



{'mcc': 0.0, 'acc': 0.20810600155884645, 'eval_loss': 1.7547448495539224}
Test Set Result: 0.20810600155884645


In [19]:
Pred=[]

countCorrect=0

for row in range(TestModel_outputs.shape[0]):
    outputs=TestModel_outputs[row]
    #print(test.iloc[row,0])
    print(outputs, end=' ')
    
    result=0
    if outputs[0]<outputs[1]:result=1
    if outputs[result]<outputs[2]:result=2
    if outputs[result]<outputs[3]:result=3
    if outputs[result]<outputs[4]:result=4
    if outputs[result]<outputs[5]:result=5
    Pred.append(result)
    print(result, ' ',test.iloc[row,1], end=' ')
    if result==test.iloc[row,1]:
        countCorrect+=1
        print('Match',countCorrect)
    print('')

print(countCorrect)

[-0.39040914  0.40200126  0.24867666  0.42444533  0.41281685  0.27566487] 3   4 
[-0.38886827  0.40101492  0.24948558  0.424803    0.41199383  0.27589822] 3   3 Match 1

[-0.3894064   0.40107957  0.25028643  0.4247478   0.41295147  0.27733898] 3   1 
[-0.38468385  0.40366676  0.2479518   0.42206803  0.4155267   0.2709958 ] 3   5 
[-0.3854295   0.4041398   0.2526069   0.4260064   0.40980104  0.2734852 ] 3   5 
[-0.3898314   0.4000115   0.25128624  0.42474115  0.41295296  0.2784948 ] 3   2 
[-0.38361967  0.40368387  0.25381744  0.4260768   0.4118292   0.266943  ] 3   4 
[-0.39467618  0.40561178  0.2630738   0.44795558  0.4086764   0.26985192] 3   5 
[-0.38973576  0.4002939   0.25049987  0.42466965  0.413418    0.27672222] 3   4 
[-0.39275095  0.40184826  0.25502577  0.42651135  0.41224512  0.27486882] 3   5 
[-0.390017    0.40485656  0.26304975  0.4488423   0.4101394   0.2681123 ] 3   5 
[-0.39090496  0.40205324  0.25188005  0.4248508   0.41400698  0.27542573] 3   3 Match 2

[-0.3957557 

[-0.3899753   0.40085983  0.24855804  0.4226682   0.41407442  0.2755294 ] 3   3 Match 22

[-0.3896599   0.4002874   0.25669554  0.42745098  0.41052085  0.278352  ] 3   1 
[-0.4074776   0.4026168   0.2603365   0.43077227  0.41716078  0.27361193] 3   2 
[-0.39188808  0.4001316   0.25055316  0.42326123  0.41665345  0.27163976] 3   1 
[-0.3892339   0.40103334  0.24878317  0.42414173  0.41279462  0.27640352] 3   2 
[-0.39948407  0.40440065  0.25878832  0.42499015  0.41212332  0.27623627] 3   5 
[-0.3882276   0.4022407   0.2482219   0.42347455  0.41241625  0.2750851 ] 3   5 
[-0.38964945  0.40119725  0.2485567   0.42437357  0.4128534   0.27537912] 3   2 
[-0.3908342   0.40113655  0.25041717  0.42416644  0.41269898  0.2767628 ] 3   0 
[-0.40097114  0.4033477   0.26147726  0.42578235  0.4114603   0.2736933 ] 3   2 
[-0.38972852  0.40112883  0.24853042  0.4224306   0.4139987   0.27583054] 3   5 
[-0.3952877   0.39979395  0.25690505  0.42688632  0.41201     0.2761032 ] 3   4 
[-0.39669424  0.404

[-0.38701835  0.40347973  0.2544971   0.42820475  0.40984276  0.26626304] 3   1 
[-0.38903728  0.40084237  0.24824607  0.42248327  0.4139321   0.2758487 ] 3   4 
[-0.389975    0.40140226  0.25050333  0.42435786  0.41202885  0.27569017] 3   2 
[-0.38982952  0.4007052   0.2483936   0.42323276  0.41411322  0.27602395] 3   1 
[-0.38916472  0.39953277  0.2521504   0.424979    0.41126582  0.27836385] 3   2 
[-0.3898474   0.40158844  0.25303057  0.42524722  0.4103985   0.27597925] 3   3 Match 48

[-0.38716304  0.40137658  0.25003448  0.42822638  0.41007388  0.27696785] 3   1 
[-0.39083153  0.40101516  0.24921721  0.42357934  0.4135654   0.27683   ] 3   3 Match 49

[-0.38902068  0.40214235  0.2576603   0.4261552   0.40830383  0.27782252] 3   0 
[-0.38949138  0.4019279   0.25249255  0.4252542   0.41146794  0.27518967] 3   3 Match 50

[-0.3900968   0.40105164  0.25103775  0.42616144  0.41095254  0.27701166] 3   1 
[-0.38704282  0.40194795  0.24815449  0.42423534  0.41182578  0.2738463 ] 3   0 
[

[-0.38920754  0.40183836  0.24847332  0.42478067  0.41115388  0.27539286] 3   2 
[-0.38966542  0.4016369   0.24946564  0.4235708   0.41302207  0.27614152] 3   0 
[-0.38803968  0.4020791   0.2509413   0.42685312  0.40939793  0.27606213] 3   1 
[-0.38983896  0.40142676  0.24971786  0.4239848   0.4134041   0.27584743] 3   5 
[-0.39326066  0.4037739   0.25860706  0.43137577  0.40997288  0.26596168] 3   3 Match 72

[-0.39036328  0.40157056  0.25230855  0.42422205  0.41053948  0.27626997] 3   4 
[-0.38883469  0.40095812  0.24943146  0.42386785  0.41327834  0.27643332] 3   1 
[-0.3884418   0.4008937   0.25361612  0.42642036  0.40954062  0.27704403] 3   1 
[-0.38922328  0.40182748  0.24785689  0.4234108   0.4124063   0.2767184 ] 3   3 Match 73

[-0.38972324  0.400168    0.250428    0.4239537   0.4129301   0.27674755] 3   4 
[-0.38943744  0.4009508   0.2487914   0.42312175  0.41333234  0.27573523] 3   1 
[-0.38649386  0.40142775  0.24974892  0.42337552  0.4121493   0.2732185 ] 3   3 Match 74

[

[-0.38912472  0.40132242  0.24888939  0.42398146  0.41231555  0.27542531] 3   1 
[-0.39082903  0.39913017  0.25223216  0.4234225   0.41256964  0.2766271 ] 3   3 Match 99

[-0.39540434  0.40390775  0.25746885  0.4298588   0.41129044  0.27368546] 3   3 Match 100

[-0.38828695  0.4014331   0.2501482   0.42160448  0.41283458  0.2754612 ] 3   4 
[-0.3854636   0.4024039   0.24660918  0.42105272  0.41426322  0.2750524 ] 3   4 
[-0.39003497  0.4004015   0.25068897  0.42479822  0.41216925  0.27606934] 3   5 
[-0.393023    0.40056303  0.25453916  0.4240813   0.4115683   0.27802438] 3   3 Match 101

[-0.39352882  0.4007529   0.2544823   0.42361864  0.4128962   0.27770868] 3   1 
[-0.38819903  0.40195972  0.24853954  0.4253151   0.4114531   0.27597445] 3   1 
[-0.39981082  0.40089774  0.25788465  0.43152955  0.41350672  0.27432334] 3   5 
[-0.38668597  0.40197533  0.2506152   0.42570022  0.410245    0.27627978] 3   2 
[-0.38959104  0.40141508  0.25072402  0.42417702  0.4115455   0.27582246] 3   3 

...

[-0.38982671  0.4007115   0.25105518  0.42555752  0.41166666  0.27683434] 3   2 
[-0.38853258  0.40037847  0.25106665  0.4249452   0.4120942   0.277658  ] 3   2 
[-0.39101228  0.40212595  0.2500815   0.42619926  0.41056886  0.2755557 ] 3   3 Match 181

[-0.3893627   0.40280858  0.24875703  0.4255798   0.41111976  0.27701616] 3   5 
[-0.3902392   0.40083024  0.24999508  0.4240507   0.41151315  0.276648  ] 3   1 
[-0.38352814  0.40255144  0.24653229  0.4220446   0.41443947  0.27118307] 3   3 Match 182

[-0.3929552   0.40318394  0.2523989   0.42660612  0.4121103   0.27359638] 3   3 Match 183

[-0.39052153  0.40197325  0.24937978  0.42335218  0.4125215   0.27561373] 3   5 
[-0.38617098  0.40342468  0.24795884  0.42648652  0.4105975   0.2767031 ] 3   2 
[-0.38977242  0.40071797  0.24914137  0.42411205  0.41379425  0.27586782] 3   1 
[-0.3894163   0.4016489   0.2493214   0.42356858  0.41248235  0.2760751 ] 3   3 Match 184

[-0.3887801   0.40211433  0.24856016  0.42453444  0.4119362   0.27600

[-0.39001235  0.40123755  0.24966964  0.42589846  0.41184977  0.27545932] 3   0 
[-0.38998717  0.40096274  0.24862424  0.42295444  0.41430712  0.27521285] 3   1 
[-0.3920279   0.40454695  0.2519864   0.42872807  0.41272134  0.2742652 ] 3   3 Match 202

[-0.38894212  0.40065342  0.24763927  0.42361665  0.4136021   0.2743207 ] 3   5 
[-0.38963416  0.40234846  0.24940667  0.42428678  0.4115178   0.2763678 ] 3   4 
[-0.39024967  0.4005214   0.25110722  0.42328677  0.4129822   0.2766541 ] 3   4 
[-0.39118135  0.40263128  0.250085    0.42363942  0.41256088  0.27701297] 3   0 
[-0.38945004  0.40120792  0.24898186  0.42456535  0.4127835   0.27654848] 3   0 
[-0.38574296  0.40359017  0.25300136  0.4282177   0.40977028  0.27510673] 3   2 
[-0.38964906  0.39981052  0.2503352   0.42497453  0.41228312  0.27661166] 3   0 
[-0.38895112  0.4014118   0.24778974  0.42372903  0.41183636  0.27699164] 3   2 
[-0.39021176  0.40100336  0.2488721   0.42334694  0.41314495  0.27623892] 3   1 
[-0.39025137  0.40

[-0.3935848   0.3991648   0.25317553  0.42565417  0.41148365  0.27758527] 3   5 
[-0.38675153  0.40106368  0.24952999  0.4246585   0.41261044  0.27696294] 3   2 
[-0.38826263  0.40188104  0.25072914  0.42490262  0.41062915  0.27603087] 3   0 
[-0.38836995  0.40125462  0.25031224  0.42205366  0.41255975  0.276327  ] 3   1 
[-0.4054398   0.40248278  0.26937607  0.42757323  0.41196015  0.2775217 ] 3   4 
[-0.38772228  0.4024996   0.2510586   0.4229035   0.41343385  0.2745574 ] 3   5 
[-0.39015847  0.4011482   0.2522498   0.4253159   0.41173068  0.27602047] 3   1 
[-0.39109293  0.40149093  0.25612718  0.42607078  0.41057763  0.27845514] 3   5 
[-0.3890913   0.40047362  0.24844185  0.42347565  0.41300392  0.27620974] 3   3 Match 225

[-0.3840959   0.40266743  0.25958386  0.4413211   0.40496188  0.26832363] 3   5 
[-0.3893227   0.40195072  0.24926028  0.42474848  0.4117812   0.27575985] 3   1 
[-0.39013678  0.40070087  0.24984217  0.42363718  0.41401297  0.27643782] 3   5 
[-0.3875856   0.40

[-0.38621172  0.40367767  0.25356674  0.42678756  0.4120887   0.26670545] 3   2 
[-0.3956816   0.40122074  0.25798118  0.43029985  0.4094344   0.27510965] 3   2 
[-0.38841546  0.401475    0.2483623   0.42463496  0.4122329   0.2761319 ] 3   2 
[-0.39001414  0.40152174  0.25319734  0.4265112   0.4114896   0.27504084] 3   2 
[-0.39236403  0.4009266   0.25331035  0.42759848  0.4092696   0.2775128 ] 3   3 Match 242

[-0.3907447   0.40127885  0.25100598  0.42556903  0.4116531   0.27647758] 3   4 
[-0.3879404   0.40185308  0.24764457  0.42443934  0.41210166  0.27578363] 3   0 
[-0.3960924   0.40451705  0.2522404   0.426805    0.41471574  0.27295342] 3   3 Match 243

[-0.38954484  0.4003014   0.24877188  0.42390394  0.41282913  0.27530771] 3   5 
[-0.38930452  0.4000927   0.25086245  0.42525992  0.41180775  0.27759704] 3   3 Match 244

[-0.38919324  0.4007437   0.25364056  0.4270336   0.41190717  0.27476606] 3   1 
[-0.39380288  0.40182644  0.25663784  0.43226075  0.41278678  0.27452084] 3   4


[-0.38992992  0.40187538  0.25016192  0.42645147  0.41004723  0.2759774 ] 3   5 
[-0.38801056  0.4021594   0.24855137  0.4259359   0.4106775   0.27667713] 3   4 
[-0.38816038  0.3998224   0.2501224   0.4244728   0.41290775  0.27625003] 3   2 
[-0.38967144  0.40199092  0.24952582  0.42371058  0.4134113   0.2745158 ] 3   3 Match 264

[-0.38763824  0.40224293  0.24741003  0.42487383  0.41201884  0.27651846] 3   3 Match 265

[-0.38847893  0.40144873  0.24867252  0.42359427  0.41259766  0.27422798] 3   4 
[-0.38867983  0.40178478  0.2492539   0.424385    0.41159588  0.27547997] 3   4 
[-0.38720357  0.40120044  0.2512594   0.424108    0.41275933  0.2752414 ] 3   2 
[-0.38991785  0.40111327  0.2485547   0.42365697  0.4129778   0.27630997] 3   3 Match 266

[-0.38914272  0.4012555   0.25267455  0.42430913  0.4129459   0.2755441 ] 3   2 
[-0.38928157  0.4028841   0.24988887  0.42529985  0.41184056  0.2751394 ] 3   5 
[-0.3880192   0.4023207   0.24670476  0.42387342  0.41186023  0.27608955] 3   

In [20]:
from sklearn import metrics
print(metrics.confusion_matrix(test['labels'],Pred))

[[  0   0   0  92   0   0]
 [  0   0   0 250   0   0]
 [  0   0   0 214   0   0]
 [  0   0   0 267   0   0]
 [  0   0   0 249   0   0]
 [  0   0   0 211   0   0]]


In [21]:
target_names = ['Pants', 'False', 'Barely-True','Hlaf-True','Mostly-True','True']

print(metrics.classification_report(test['labels'], Pred,target_names =target_names))

              precision    recall  f1-score   support

       Pants       0.00      0.00      0.00        92
       False       0.00      0.00      0.00       250
 Barely-True       0.00      0.00      0.00       214
   Hlaf-True       0.21      1.00      0.34       267
 Mostly-True       0.00      0.00      0.00       249
        True       0.00      0.00      0.00       211

    accuracy                           0.21      1283
   macro avg       0.03      0.17      0.06      1283
weighted avg       0.04      0.21      0.07      1283



C:\Users\Mark\Anaconda3\envs\thesis\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
# saving the output of the models to CSVs
#these are 1X6 classification vectors

SavesDirectory='./TunedModels/'+model_class+'/'+model_version+"/Saves/"
print('Saving...')
trainOut = pd.DataFrame(data= TrainModel_outputs )
trainOut.to_csv(SavesDirectory+'trainOut.tsv', sep='\t',  index=False)

evalOut = pd.DataFrame(data= EvalModel_outputs )
evalOut.to_csv(SavesDirectory+'evalOut.tsv', sep='\t',  index=False)

testOut = pd.DataFrame(data= TestModel_outputs )
testOut.to_csv(SavesDirectory+'testOut.tsv', sep='\t',  index=False)

print('Saving Complete on',datetime.now() ,'in:', SavesDirectory)

Saving...
Saving Complete on 2020-08-19 15:18:14.696782 in: ./TunedModels/roberta/roberta-large/Saves/


In [28]:
del(model)
#del(train,Eval,test)
del(trainOut,evalOut,testOut)
torch.cuda.empty_cache()

#  Adding the reputation vector

This section takes the output results from the transformer used above and uses it together with the speaker's reputation to enhance the classification.

Before running this section it is suggested that you halt the program and start running it again from this cell. The neural net will likely have an error caused by some unreleased variable used by thr simple transformers library. 

In [1]:
import pandas as pd
import torch
import torch.nn as nn
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [5]:

train=pd.read_excel('trainReputation.xlsx' )
train=train.iloc[:,:-2].astype(float)
train=train/200  #for scaling
#train

model_class='roberta'  # bert or roberta or albert
model_version='roberta-large' #bert-base-cased, roberta-base, roberta-large, albert-base-v2 OR albert-large-v2

SavesDirectory='./TunedModels/'+model_class+'/'+model_version+"/Saves/"
TF_Output=pd.read_csv( SavesDirectory+'trainOut.tsv', sep='\t')

train=pd.concat([train,TF_Output], axis=1)

train

,PantsTotal,NotRealTotal,BarelyTotal,HalfTotal,MostlyTotal,Truths,0,1,2,3,4,5
0,0.005,0.000,0.000,0.000,0.000,0.000,-0.387377,0.400232,0.249287,0.425540,0.413897,0.277364
1,0.095,0.160,0.170,0.290,0.165,0.165,-0.388868,0.401241,0.248872,0.422384,0.414152,0.275785
2,0.005,0.010,0.005,0.015,0.040,0.010,-0.389275,0.401526,0.248285,0.423318,0.413382,0.275590
3,0.005,0.010,0.005,0.015,0.040,0.010,-0.389653,0.401985,0.249432,0.422432,0.413908,0.275714
4,0.035,0.145,0.200,0.345,0.380,0.365,-0.390197,0.401123,0.249028,0.424097,0.413163,0.275274
...,...,...,...,...,...,...,...,...,...,...,...,...
10264,0.005,0.030,0.070,0.050,0.050,0.020,-0.388905,0.401978,0.248246,0.424185,0.411216,0.275639
10265,0.055,0.075,0.080,0.100,0.050,0.035,-0.390886,0.401707,0.255645,0.426733,0.411876,0.274773
10266,0.035,0.115,0.140,0.190,0.170,0.075,-0.388285,0.400249,0.250624,0.424685,0.412110,0.277521
10267,0.305,0.570,0.315,0.255,0.185,0.070,-0.390006,0.401340,0.250346,0.424074,0.412756,0.276400


In [6]:
TrainLables=pd.read_excel('trainReputation.xlsx' )
TrainLables=TrainLables.iloc[:,-1] 

TrainLables=pd.get_dummies(TrainLables)
TrainLables

,0,1,2,3,4,5
0,1,0,0,0,0,0
1,0,0,0,1,0,0
2,0,0,0,0,1,0
3,0,0,0,0,1,0
4,0,0,0,0,0,1
...,...,...,...,...,...,...
10264,0,0,0,0,1,0
10265,0,0,0,0,0,1
10266,0,0,0,1,0,0
10267,0,1,0,0,0,0


In [7]:
input=torch.tensor(train.values)
del(train)
input

tensor([[0.0050, 0.0000, 0.0000,  ..., 0.4255, 0.4139, 0.2774],
        [0.0950, 0.1600, 0.1700,  ..., 0.4224, 0.4142, 0.2758],
        [0.0050, 0.0100, 0.0050,  ..., 0.4233, 0.4134, 0.2756],
        ...,
        [0.0350, 0.1150, 0.1400,  ..., 0.4247, 0.4121, 0.2775],
        [0.3050, 0.5700, 0.3150,  ..., 0.4241, 0.4128, 0.2764],
        [0.0000, 0.0050, 0.0000,  ..., 0.4278, 0.4118, 0.2721]],
       dtype=torch.float64)

In [8]:
targets=torch.tensor(TrainLables.astype(float).values)
del(TrainLables)
targets

tensor([[1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        ...,
        [0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.]], dtype=torch.float64)

In [9]:
 
size= torch.tensor(input[0].size())
InputSize=size.item()

OutputSize=torch.tensor(targets[0].size()).item()

print('input size:', InputSize)
print('output size:', OutputSize)

input size: 12
output size: 6


In [10]:

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        
         
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(InputSize, 24)  # input size 
        self.fc2 = nn.Linear(24, 12)
        self.fc3 = nn.Linear(12, OutputSize)  #classifies 'outputsize' different classes

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x)) 
        x = torch.tanh(self.fc3(x)).double()
        return x

    

#now we use it

net = Net()

In [11]:
#### here we  setup the neural network parameters
# pick an optimizer (Simple Gradient Descent)

learning_rate = 9e-4
criterion = nn.MSELoss()  #computes the loss Function

import torch.optim as optim

# creating optimizer
#optimizer = optim.SGD(net.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)


In [12]:
for epoch in range(500):  
        
    optimizer.zero_grad()   # zero the gradient buffers
    output = net(input.float())

    loss = criterion(output, targets)
    print('Loss:', loss, ' at epoch:', epoch)

    loss.backward()  #backprop
    optimizer.step()    # Does the update

Loss: tensor(0.2403, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 0
Loss: tensor(0.2355, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 1
Loss: tensor(0.2309, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2
Loss: tensor(0.2265, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 3
Loss: tensor(0.2221, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 4
Loss: tensor(0.2179, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 5
Loss: tensor(0.2138, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 6
Loss: tensor(0.2099, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7
Loss: tensor(0.2060, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 8
Loss: tensor(0.2023, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 9
Loss: tensor(0.1988, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 10
Loss: tensor(0.1953, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 11
Loss: tensor(0

Loss: tensor(0.1370, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 105
Loss: tensor(0.1370, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 106
Loss: tensor(0.1370, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 107
Loss: tensor(0.1370, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 108
Loss: tensor(0.1369, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 109
Loss: tensor(0.1369, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 110
Loss: tensor(0.1369, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 111
Loss: tensor(0.1369, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 112
Loss: tensor(0.1369, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 113
Loss: tensor(0.1369, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 114
Loss: tensor(0.1369, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 115
Loss: tensor(0.1369, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epo

Loss: tensor(0.1363, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 217
Loss: tensor(0.1363, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 218
Loss: tensor(0.1363, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 219
Loss: tensor(0.1363, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 220
Loss: tensor(0.1363, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 221
Loss: tensor(0.1363, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 222
Loss: tensor(0.1363, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 223
Loss: tensor(0.1363, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 224
Loss: tensor(0.1363, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 225
Loss: tensor(0.1363, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 226
Loss: tensor(0.1363, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 227
Loss: tensor(0.1363, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epo

Loss: tensor(0.1357, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 331
Loss: tensor(0.1357, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 332
Loss: tensor(0.1357, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 333
Loss: tensor(0.1357, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 334
Loss: tensor(0.1357, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 335
Loss: tensor(0.1357, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 336
Loss: tensor(0.1357, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 337
Loss: tensor(0.1357, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 338
Loss: tensor(0.1357, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 339
Loss: tensor(0.1357, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 340
Loss: tensor(0.1357, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 341
Loss: tensor(0.1357, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epo

Loss: tensor(0.1355, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 444
Loss: tensor(0.1355, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 445
Loss: tensor(0.1355, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 446
Loss: tensor(0.1355, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 447
Loss: tensor(0.1355, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 448
Loss: tensor(0.1355, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 449
Loss: tensor(0.1355, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 450
Loss: tensor(0.1355, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 451
Loss: tensor(0.1355, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 452
Loss: tensor(0.1355, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 453
Loss: tensor(0.1355, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 454
Loss: tensor(0.1355, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epo

...

Loss: tensor(0.1011, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7474
Loss: tensor(0.1011, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7475
Loss: tensor(0.1011, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7476
Loss: tensor(0.1011, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7477
Loss: tensor(0.1011, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7478
Loss: tensor(0.1011, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7479
Loss: tensor(0.1011, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7480
Loss: tensor(0.1011, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7481
Loss: tensor(0.1011, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7482
Loss: tensor(0.1011, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7483
Loss: tensor(0.1011, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7484
Loss: tensor(0.1011, dtype=torch.float64, grad_fn=<MseLossBackwar

Loss: tensor(0.1010, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7595
Loss: tensor(0.1010, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7596
Loss: tensor(0.1010, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7597
Loss: tensor(0.1010, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7598
Loss: tensor(0.1010, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7599
Loss: tensor(0.1010, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7600
Loss: tensor(0.1010, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7601
Loss: tensor(0.1010, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7602
Loss: tensor(0.1010, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7603
Loss: tensor(0.1010, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7604
Loss: tensor(0.1010, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7605
Loss: tensor(0.1010, dtype=torch.float64, grad_fn=<MseLossBackwar

Loss: tensor(0.1009, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7714
Loss: tensor(0.1009, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7715
Loss: tensor(0.1009, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7716
Loss: tensor(0.1009, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7717
Loss: tensor(0.1009, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7718
Loss: tensor(0.1009, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7719
Loss: tensor(0.1009, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7720
Loss: tensor(0.1009, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7721
Loss: tensor(0.1009, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7722
Loss: tensor(0.1009, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7723
Loss: tensor(0.1009, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7724
Loss: tensor(0.1009, dtype=torch.float64, grad_fn=<MseLossBackwar

Loss: tensor(0.1009, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7834
Loss: tensor(0.1009, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7835
Loss: tensor(0.1009, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7836
Loss: tensor(0.1009, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7837
Loss: tensor(0.1009, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7838
Loss: tensor(0.1009, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7839
Loss: tensor(0.1009, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7840
Loss: tensor(0.1009, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7841
Loss: tensor(0.1009, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7842
Loss: tensor(0.1009, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7843
Loss: tensor(0.1009, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7844
Loss: tensor(0.1009, dtype=torch.float64, grad_fn=<MseLossBackwar

Loss: tensor(0.1008, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7953
Loss: tensor(0.1008, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7954
Loss: tensor(0.1008, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7955
Loss: tensor(0.1008, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7956
Loss: tensor(0.1008, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7957
Loss: tensor(0.1008, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7958
Loss: tensor(0.1008, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7959
Loss: tensor(0.1008, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7960
Loss: tensor(0.1008, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7961
Loss: tensor(0.1008, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7962
Loss: tensor(0.1008, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7963
Loss: tensor(0.1008, dtype=torch.float64, grad_fn=<MseLossBackwar

In [20]:
#save the FCNN model

stage='NNetwork/'
SavesDirectory='./TunedModels/'+model_class+'/'+model_version+"/"+stage
#PATH = SavesDirectory+'Tanh_MSE_adam4645.pth'

torch.save(net.state_dict(), PATH)

# more on saving pytorch networks: https://pytorch.org/docs/stable/notes/serialization.html

In [26]:
#load previously saved FCNN model 

stage='NNetwork/'
SavesDirectory='./TunedModels/'+model_class+'/'+model_version+"/"+stage
#PATH = SavesDirectory+'Tanh_MSE_adam21.pth'

net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [14]:
# load the test data

TestData=pd.read_excel('testReputation.xlsx' )
TestData=TestData.iloc[:,:-2].astype(float)
TestData=TestData/200

SavesDirectory='./TunedModels/'+model_class+'/'+model_version+"/Saves/"
TF_Output=pd.read_csv( SavesDirectory+'testOut.tsv', sep='\t')

TestData=pd.concat([TestData,TF_Output], axis=1)


TestData=torch.tensor(TestData.values)
TestData

tensor([[0.0050, 0.0100, 0.0050,  ..., 0.4244, 0.4128, 0.2757],
        [0.0000, 0.0000, 0.0000,  ..., 0.4248, 0.4120, 0.2759],
        [0.0200, 0.0250, 0.0600,  ..., 0.4247, 0.4130, 0.2773],
        ...,
        [0.0100, 0.0050, 0.0250,  ..., 0.4227, 0.4129, 0.2743],
        [0.2200, 0.0950, 0.0350,  ..., 0.4266, 0.4102, 0.2756],
        [0.0050, 0.0600, 0.0100,  ..., 0.4263, 0.4101, 0.2760]],
       dtype=torch.float64)

In [15]:
labels=pd.read_excel('testReputation.xlsx' )
labels=labels.iloc[:,-1] 
labelsOneHot=pd.get_dummies(labels)

TestLables =torch.tensor(labelsOneHot.values)
TestLables

tensor([[0, 0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0, 0],
        ...,
        [0, 0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0]], dtype=torch.uint8)

In [16]:

correct = 0
total = 0

Y=[]  #target
Pred=[]  #predicted

with torch.no_grad():
    for row in range(len(TestData)):
        outputs = net(TestData[row,:].float())
        result=0
        total+=1
        if outputs[0]<outputs[1]:result=1
        if outputs[result]<outputs[2]:result=2
        if outputs[result]<outputs[3]:result=3
        if outputs[result]<outputs[4]:result=4
        if outputs[result]<outputs[5]:result=5
        
        if labelsOneHot.iloc[row,result]==1: correct+=1
        
        Y.append(result)
        Pred.append(labels.iloc[row])
        
        print(result, end=' ')
        
       
print('Correct:', correct, 'out of:', total )
print('Accuracy of the network : ',( 100 * correct / total))

4 3 2 1 1 1 3 4 4 4 4 3 3 4 3 3 1 3 1 4 4 4 3 1 1 4 1 3 3 4 4 1 0 4 4 1 4 4 4 4 3 1 4 1 4 4 3 4 4 4 4 1 1 3 3 4 5 5 4 1 1 1 4 3 1 4 3 3 4 4 1 4 1 1 1 1 1 1 4 4 1 1 1 4 1 1 4 4 1 4 4 4 3 1 2 4 4 1 3 1 4 0 4 3 3 1 1 3 1 3 1 3 3 3 1 1 0 4 1 2 2 4 4 3 1 1 1 1 5 1 3 2 3 1 1 2 0 2 2 1 4 1 3 3 3 4 4 3 4 3 0 1 1 3 5 3 4 5 3 3 3 1 4 4 3 5 4 2 3 1 3 3 4 5 3 1 3 4 0 2 1 3 4 5 1 3 2 4 1 3 5 2 4 0 1 3 2 2 3 1 1 4 5 3 4 3 3 5 1 3 3 3 3 1 4 1 2 3 3 1 1 1 4 1 1 5 3 2 3 0 5 1 0 1 1 5 4 0 4 2 2 1 2 4 4 1 2 1 3 3 0 3 4 1 3 3 4 0 4 1 2 1 1 2 0 3 1 3 4 2 3 4 1 2 5 2 5 1 2 4 2 1 4 5 1 4 3 1 5 0 3 5 1 3 1 1 3 4 2 3 1 5 3 3 5 3 2 5 3 1 1 3 1 3 3 4 5 3 3 3 4 1 5 1 3 0 5 1 5 4 4 3 3 3 5 1 1 3 3 2 4 2 3 3 3 5 2 5 5 1 1 0 4 1 4 1 1 3 1 2 1 1 1 1 2 0 3 5 2 1 0 1 4 4 4 3 3 5 2 1 3 1 3 4 3 3 3 3 1 4 3 2 1 1 2 1 3 4 3 4 4 3 0 1 4 5 2 3 3 3 4 2 4 1 2 1 1 5 3 3 4 4 4 1 1 3 4 3 4 3 5 1 2 1 2 5 3 4 4 1 5 4 4 1 3 1 3 2 2 3 1 3 1 1 4 3 4 3 1 1 5 1 3 4 4 3 1 3 1 3 3 4 1 0 3 3 3 1 3 3 2 3 1 3 4 1 2 4 3 1 1 3 5 3 3 3 3 4 3 4 

In [17]:
from sklearn import metrics 
print(metrics.confusion_matrix(Y,Pred))

[[ 46  14   5   1   2   3]
 [ 21 134  38  42  36  35]
 [  6  27  80  18  13  13]
 [ 13  39  42 140  59  26]
 [  4  26  35  54 129  67]
 [  2  10  14  12  10  67]]


In [18]:
target_names = ['Pants', 'False', 'Barely-True','Half-True','Mostly-True','True']

print(metrics.classification_report(Y, Pred,target_names =target_names))

              precision    recall  f1-score   support

       Pants       0.50      0.65      0.56        71
       False       0.54      0.44      0.48       306
 Barely-True       0.37      0.51      0.43       157
   Half-True       0.52      0.44      0.48       319
 Mostly-True       0.52      0.41      0.46       315
        True       0.32      0.58      0.41       115

    accuracy                           0.46      1283
   macro avg       0.46      0.50      0.47      1283
weighted avg       0.49      0.46      0.47      1283

